In [5]:
import os
import math
from docplex.mp.model import Model
from docplex.mp.relax_linear import LinearRelaxer

def fill_variables(file, n_customers, n_available_depots, coordinates_depots, coordinates_customers, v_capacity, capacities_depots,
        customers_demands, opening_costs_depots, opening_cost_route, zero_or_not):
    
    line = next(file)
    n_customers = int(line.strip())
    print("Número de customers: " + str(n_customers))
    line = next(file)
    n_available_depots = int(line.strip())
    print("Número de depots disponibles: " + str(n_available_depots))
    line = next(file)
    line = next(file)
            
    #coordenadas depots
    while(line.strip()):
        coordinates_depots.append(line.split())    
        line = next(file)
            
    #print("Coordenadas depots:")
    #print(coordinates_depots)
            
    line = next(file)
            
        #Coordenadas customers
    while(line.strip()):
        coordinates_customers.append(line.split())
                
        line = next(file)
            
    #print("Coordenadas customers:")
    #print(coordinates_customers)
            
    line = next(file)
            
    #Capacidad de vehículos
    v_capacity = int(line.strip())
    #print("Capacidad vehículos: " + v_capacity)
    line = next(file)
    line = next(file)
            
    #Capacidad de los depots
    while(line.strip()):
        capacities_depots.append(line.split())
                
        line = next(file)
            
    #print("Capacidades depots:")
    #print(capacities_depots)
            
    line = next(file)
            
    while(line.strip()):
        customers_demands.append(line.strip())
                
        line = next(file)
            
    #print("Demandas customers:")
    #print(customers_demands)
            
    line = next(file)
            
    while(line.strip()):
        opening_costs_depots.append(line.split())
                
        line = next(file)
            
    #print("Costos depots:")
    #print(opening_costs_depots)
            
    line = next(file)
            
            
    opening_cost_route = int(line.strip())
                
    line = next(file)
            
    #print("opening cost route:")
    #print(opening_cost_route)
            
    line = next(file)
    zero_or_not = int(line.strip())
            
    #print("zero o uno: " + zero_or_not)
    return [n_customers, n_available_depots, v_capacity, opening_cost_route, zero_or_not]
    
def distance_cost(a1, a2, b1, b2):
    a1 = int(a1)
    a2 = int(a2)
    b1 = int(b1)
    b2 = int(b2)
    x1 = a1-b1
    x2 = a2-b2
    x1 = pow(x1, 2)
    x2 = pow(x2, 2)
    return math.sqrt(x1 + x2)

In [13]:
file_path = "Inst1"
for filename in os.listdir(file_path):
    print("\nSiguiente archivo\n")
    f = os.path.join(file_path, filename)
    # checking if it is a file
    if os.path.isfile(f):
        
        #Contenedores de info inicializados en nada
        n_customers = 0
        n_available_depots = 0
        coordinates_depots = []
        coordinates_customers = []
        v_capacity = 0
        capacities_depots = []
        customers_demands = []
        opening_costs_depots = []
        opening_cost_route = 0
        zero_or_not = 1
        
        #Acá se toma la info de los archivos .dat
        with open(f, 'r') as file:
            lista_var = []
            lista_var = fill_variables(file, n_customers, n_available_depots, coordinates_depots, coordinates_customers, 
                                       v_capacity, capacities_depots, customers_demands, opening_costs_depots, 
                                       opening_cost_route, zero_or_not)
            n_customers = lista_var[0]
            n_available_depots = lista_var[1]
            v_capacity = lista_var[2]
            opening_cost_route = lista_var[3]
            zero_or_not = lista_var[4]
        
            
        #Empezar operaciones acá
        #distancias_d2d = []  VIAJE ENTRE 2 DEPOTS NO PERMITIDO
        distancias_d2c = []
        distancias_c2c = []
        distancias_c2d = []
        distancias = []
        
        #Acá se consiguen los c_ij (costos de caminos)
        for i in coordinates_depots:
            '''for j in coordinates_depots:
                if j==i:
                    continue
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_d2d.append(distance_cost(i[0], i[1], j[0], j[1]))'''
            for j in coordinates_customers:
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_d2c.append(distance_cost(i[0], i[1], j[0], j[1]))
        for i in coordinates_customers:
            for j in coordinates_depots:
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_c2d.append(distance_cost(i[0], i[1], j[0], j[1]))
        for i in coordinates_customers:
            for j in coordinates_customers:
                if j==i:
                    continue
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_c2c.append(distance_cost(i[0], i[1], j[0], j[1]))

        #MODELO
        mdl = Model(name="MDRP")
        x = mdl.binary_var_list(len(distancias), name="x")
        mdl.print_information()
        
        #print(len(distancias))
        #C_ij * X_ij
        ab = [distancias[i]*x[i] for i in range(n_customers*n_available_depots)]
        mdl.minimize(mdl.sum(ab))

        #X_dj = yd
        for i in range(n_available_depots):
            offset = i*n_customers
            xdj = []
            for j in range(offset, offset+n_customers):
                xdj.append(distancias[j]*x[j])
            mdl.add_constraint(sum(xdj) <= 1)
            mdl.add_constraint(sum(xdj) >= 0)
            
        #X_jd = yd
        for i in range(n_available_depots):
            xjd = []
            for j in range(n_customers):
                offset = j*n_available_depots + n_available_depots*n_customers
                #print(str(offset) + " vs " + str(len(distancias)) + " vs " + str(len(x)))
                xjd.append(distancias[offset]*x[offset])
            mdl.add_constraint(sum(xdj) <= 1)
            mdl.add_constraint(sum(xdj) >= 0)
            
        
            
        


Siguiente archivo

Número de customers: 100
Número de depots disponibles: 10
Model: MDRP
 - number of variables: 11898
   - binary=11898, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
29.732x_1000+30.806x_1010+27.074x_1020+25.080x_1030+20x_1040+46.228x_1050+4.243x_1060+28.231x_1070+36.222x_1080+27.019x_1090+51.624x_1100+37.656x_1110+12.042x_1120+25.807x_1130+17.464x_1140+10.440x_1150+5x_1160+46.043x_1170+35.693x_1180+39.661x_1190+57.801x_1200+48.094x_1210+33.541x_1220+42.059x_1230+41.195x_1240+41.195x_1250+47.381x_1260+45.277x_1270+43.186x_1280+22.023x_1290+30.610x_1300+55.227x_1310+43.600x_1320+28.862x_1330+24.739x_1340+34x_1350+38x_1360+17.117x_1370+27.203x_1380+39.623x_1390+36.056x_1400+5.657x_1410+47.634x_1420+24.839x_1430+19.313x_1440+25.962x_1450+17.088x_1460+29.155x_1470+8.062x_1480+18.028x_1490+40.112x_1500+18.248x_1510+38.328x_1520+22.804x_1530+53.824x_1540+49.518x_1550+60.605x_1560+41.110

Model: MDRP
 - number of variables: 11898
   - binary=11898, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
29.732x_1000+30.806x_1010+27.074x_1020+25.080x_1030+20x_1040+46.228x_1050+4.243x_1060+28.231x_1070+36.222x_1080+27.019x_1090+51.624x_1100+37.656x_1110+12.042x_1120+25.807x_1130+17.464x_1140+10.440x_1150+5x_1160+46.043x_1170+35.693x_1180+39.661x_1190+57.801x_1200+48.094x_1210+33.541x_1220+42.059x_1230+41.195x_1240+41.195x_1250+47.381x_1260+45.277x_1270+43.186x_1280+22.023x_1290+30.610x_1300+55.227x_1310+43.600x_1320+28.862x_1330+24.739x_1340+34x_1350+38x_1360+17.117x_1370+27.203x_1380+39.623x_1390+36.056x_1400+5.657x_1410+47.634x_1420+24.839x_1430+19.313x_1440+25.962x_1450+17.088x_1460+29.155x_1470+8.062x_1480+18.028x_1490+40.112x_1500+18.248x_1510+38.328x_1520+22.804x_1530+53.824x_1540+49.518x_1550+60.605x_1560+41.110x_1570+30.083x_1580+28.018x_1590+42.579x_1600+7.071x_1610+28.018x_1620+47.011x

Model: MDRP
 - number of variables: 11896
   - binary=11896, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
37.216x_1000+23.259x_1010+22.361x_1020+30.480x_1030+43.909x_1040+40.224x_1050+24.021x_1060+45.122x_1070+39.623x_1080+39.560x_1090+42.190x_1100+38.897x_1110+24.187x_1120+32.757x_1130+43.566x_1140+31.321x_1150+43.863x_1160+36.235x_1170+41.881x_1180+22.023x_1190+37x_1200+42.012x_1210+42.544x_1220+31.016x_1230+32.558x_1240+21.024x_1250+41.110x_1260+25.456x_1270+49.244x_1280+29.833x_1290+35.805x_1300+25.318x_1310+46.615x_1320+26.077x_1330+42.059x_1340+33.015x_1350+43.566x_1360+21.471x_1370+28.071x_1380+34.670x_1390+39.446x_1400+28.320x_1410+40.311x_1420+37.014x_1430+45.618x_1440+38.013x_1450+21.095x_1460+38.601x_1470+17.029x_1480+37.363x_1490+41.110x_1500+38.275x_1510+32.016x_1520+49.769x_1530+26.420x_1540+32.140x_1550+27.731x_1560+47.854x_1570+47.011x_1580+26.019x_1590+36.125x_1600+44.283x_1610+23.

Model: MDRP
 - number of variables: 11896
   - binary=11896, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
37.216x_1000+23.259x_1010+22.361x_1020+30.480x_1030+43.909x_1040+40.224x_1050+24.021x_1060+45.122x_1070+39.623x_1080+39.560x_1090+42.190x_1100+38.897x_1110+24.187x_1120+32.757x_1130+43.566x_1140+31.321x_1150+43.863x_1160+36.235x_1170+41.881x_1180+22.023x_1190+37x_1200+42.012x_1210+42.544x_1220+31.016x_1230+32.558x_1240+21.024x_1250+41.110x_1260+25.456x_1270+49.244x_1280+29.833x_1290+35.805x_1300+25.318x_1310+46.615x_1320+26.077x_1330+42.059x_1340+33.015x_1350+43.566x_1360+21.471x_1370+28.071x_1380+34.670x_1390+39.446x_1400+28.320x_1410+40.311x_1420+37.014x_1430+45.618x_1440+38.013x_1450+21.095x_1460+38.601x_1470+17.029x_1480+37.363x_1490+41.110x_1500+38.275x_1510+32.016x_1520+49.769x_1530+26.420x_1540+32.140x_1550+27.731x_1560+47.854x_1570+47.011x_1580+26.019x_1590+36.125x_1600+44.283x_1610+23.

Model: MDRP
 - number of variables: 11886
   - binary=11886, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
42.190x_1000+31.623x_1010+33.242x_1020+26.019x_1030+43.012x_1040+38.079x_1050+36.014x_1060+33.541x_1070+28.018x_1080+37.643x_1090+27.074x_1100+34.366x_1110+32.249x_1120+36.139x_1130+27.203x_1140+44.407x_1150+37.014x_1160+43.012x_1170+36.056x_1180+36.688x_1190+32.249x_1200+38.601x_1210+26x_1220+31.145x_1230+46.098x_1240+37.014x_1250+30.806x_1260+37.014x_1270+34.928x_1280+43.105x_1290+14.213x_1300+2x_1310+12.369x_1320+11.402x_1330+x_1340+11.705x_1350+12.207x_1360+9.055x_1370+15.652x_1380+16.553x_1390+12x_1400+5.385x_1410+14.866x_1420+9.220x_1430+13.928x_1440+9x_1450+5.099x_1460+6.708x_1470+17x_1480+8.544x_1490+7x_1500+17.720x_1510+13.038x_1520+12.806x_1530+11.402x_1540+11.402x_1550+10.050x_1560+3.606x_1570+10.198x_1580+9.220x_1590+42.720x_1600+42.202x_1610+26.627x_1620+38.601x_1630+38.601x_1640+3

...

Model: MDRP
 - number of variables: 43780
   - binary=43780, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
18.385x_2000+14.560x_2010+16.401x_2020+24.698x_2030+19.723x_2040+15.620x_2050+20.224x_2060+25.080x_2070+11.402x_2080+19x_2090+15.556x_2100+16.279x_2110+9.487x_2120+24.083x_2130+13.892x_2140+26.077x_2150+14.213x_2160+11.662x_2170+8.944x_2180+21.840x_2190+22.804x_2200+21.932x_2210+11.180x_2220+26.683x_2230+25.080x_2240+13.601x_2250+20.100x_2260+16.279x_2270+22.091x_2280+28.792x_2290+10.198x_2300+18.868x_2310+13.038x_2320+15.133x_2330+26.476x_2340+24.187x_2350+17.263x_2360+12.166x_2370+18.788x_2380+15.652x_2390+26.420x_2400+16.279x_2410+18.028x_2420+24.083x_2430+25.495x_2440+23.087x_2450+10.050x_2460+13.038x_2470+12.530x_2480+15.232x_2490+24.042x_2500+26.926x_2510+15.652x_2520+19.313x_2530+14.866x_2540+23x_2550+16.553x_2560+24.352x_2570+21.541x_2580+26.476x_2590+23.195x_2600+12x_2610+12.207x_2620+

Model: MDRP
 - number of variables: 43780
   - binary=43780, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
18.385x_2000+14.560x_2010+16.401x_2020+24.698x_2030+19.723x_2040+15.620x_2050+20.224x_2060+25.080x_2070+11.402x_2080+19x_2090+15.556x_2100+16.279x_2110+9.487x_2120+24.083x_2130+13.892x_2140+26.077x_2150+14.213x_2160+11.662x_2170+8.944x_2180+21.840x_2190+22.804x_2200+21.932x_2210+11.180x_2220+26.683x_2230+25.080x_2240+13.601x_2250+20.100x_2260+16.279x_2270+22.091x_2280+28.792x_2290+10.198x_2300+18.868x_2310+13.038x_2320+15.133x_2330+26.476x_2340+24.187x_2350+17.263x_2360+12.166x_2370+18.788x_2380+15.652x_2390+26.420x_2400+16.279x_2410+18.028x_2420+24.083x_2430+25.495x_2440+23.087x_2450+10.050x_2460+13.038x_2470+12.530x_2480+15.232x_2490+24.042x_2500+26.926x_2510+15.652x_2520+19.313x_2530+14.866x_2540+23x_2550+16.553x_2560+24.352x_2570+21.541x_2580+26.476x_2590+23.195x_2600+12x_2610+12.207x_2620+

Model: MDRP
 - number of variables: 2950
   - binary=2950, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
22.023x_250+9.055x_255+7.616x_260+28.160x_265+27.203x_270+46.325x_275+44.011x_280+30.150x_285+34.059x_290+25.179x_295+24.187x_300+22.361x_305+41.400x_310+21.024x_320+5.657x_325+11.402x_330+38.275x_335+11x_340+27.074x_345+25.612x_350+42.048x_355+37.108x_360+52.154x_365+31.257x_370+18.028x_375+51.662x_380+36.674x_385+42.202x_390+18.111x_395+38.639x_400+42.438x_405+49.396x_410+4.472x_415+32.140x_420+43.012x_425+24.698x_430+32.757x_435+7.616x_440+55.973x_445+19.235x_450+58.052x_455+29.547x_460+44.181x_465+25x_470+34.482x_475+16.763x_480+25.456x_485+30.083x_490+14.142x_495 <= 1
22.023x_250+9.055x_255+7.616x_260+28.160x_265+27.203x_270+46.325x_275+44.011x_280+30.150x_285+34.059x_290+25.179x_295+24.187x_300+22.361x_305+41.400x_310+21.024x_320+5.657x_325+11.402x_330+38.275x_335+11x_340+27.074x_345+25.612

Model: MDRP
 - number of variables: 2948
   - binary=2948, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
29.732x_250+26.077x_255+16.279x_260+25x_265+29.017x_270+30x_275+38.471x_280+27.019x_285+43.658x_290+43.863x_295+36.014x_300+35.693x_305+36.878x_310+37.443x_315+26.833x_320+31.765x_325+30.806x_330+26.926x_335+34.886x_340+35.228x_345+30.150x_350+32.140x_355+30.017x_360+37.577x_365+38x_370+21.213x_375+21.378x_380+43.658x_385+31.780x_390+44.102x_395+41.773x_400+44.643x_405+41.195x_410+28.178x_415+23.409x_420+25.962x_425+39.812x_430+40.050x_435+26.173x_440+32.650x_445+22.561x_450+44.553x_455+31x_460+39.812x_465+37.121x_470+37.483x_475+22.023x_480+24.042x_485+27.019x_490+27.459x_495 <= 1
29.732x_250+26.077x_255+16.279x_260+25x_265+29.017x_270+30x_275+38.471x_280+27.019x_285+43.658x_290+43.863x_295+36.014x_300+35.693x_305+36.878x_310+37.443x_315+26.833x_320+31.765x_325+30.806x_330+26.926x_335+34.886x_34

Model: MDRP
 - number of variables: 2950
   - binary=2950, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP
20.616x_250+33.526x_255+18.974x_260+37.855x_265+33.242x_270+30.871x_275+30.265x_280+19.698x_285+29.155x_290+38.275x_295+23.259x_300+37.855x_305+20.591x_310+23.537x_315+28.443x_320+3.606x_325+2.828x_330+4.123x_335+13.038x_340+7.071x_345+x_350+8.062x_355+7.071x_360+4.243x_365+9.220x_370+13.153x_375+10x_380+7.211x_385+7x_390+7.280x_395+47.202x_400+40.311x_405+27.019x_410+25.179x_415+37.014x_420+24.413x_425+16.155x_430+40.817x_435+16.031x_440+27.459x_445+32.757x_450+18.788x_455+38.079x_460+14.142x_465+26.627x_470+15x_475+21.024x_480+21.401x_485+26.173x_490+38.210x_495 <= 1
20.616x_250+33.526x_255+18.974x_260+37.855x_265+33.242x_270+30.871x_275+30.265x_280+19.698x_285+29.155x_290+38.275x_295+23.259x_300+37.855x_305+20.591x_310+23.537x_315+28.443x_320+3.606x_325+2.828x_330+4.123x_335+13.038x_340+7.071x